In [2]:
import psycopg2 as ps
import pandas as pd

In [3]:
df = pd.read_csv('data.csv', index_col=0) 
display(df)

,opponent,div_rank,gb,attendance,cLI,winning_percentage,year,total_games_played,weekday,month,night_game,al_east_opp
date,,,,,,,,,,,,
"Monday, Apr 11",MIN,4,-2.5,57806,0.78,0.167,1994,5,1,Apr,1,0
"Tuesday, Apr 12",MIN,4,-2.0,13489,0.86,0.286,1994,6,1,Apr,1,0
"Wednesday, Apr 13",MIN,4,-3.0,16207,0.91,0.250,1994,7,1,Apr,0,0
"Friday, Apr 15",MIL,4,-2.0,16969,0.86,0.333,1994,8,0,Apr,1,0
"Saturday, Apr 16",MIL,4,-2.5,32990,0.92,0.300,1994,9,0,Apr,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
"Wednesday, Jul 7",NYY,3,-9.0,17205,0.93,0.517,2021,86,1,Jul,1,1
"Thursday, Jul 8",NYY,3,-8.0,17524,0.83,0.523,2021,87,1,Jul,0,1
"Friday, Jul 9",LAA,3,-7.0,20381,0.93,0.528,2021,88,0,Jul,1,0


In [4]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS date (
                    date TEXT NOT NULL PRIMARY KEY,
                    opponent TEXT NOT NULL,
                    div_rank INTEGER NOT NULL,
                    gb INTEGER NOT NULL,
                    attendance INTEGER NOT NULL,
                    cLI FLOAT NOT NULL,
                    winning_percentage FLOAT NOT NULL,
                    year INTEGER NOT NULL,
                    total_games_played INTEGER NOT NULL,
                    weekday INTEGER NOT NULL,
                    month TEXT NOT NULL,
                    night_game INTEGER NOT NULL,
                    al_east_opp INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, opponent,div_rank,gb,attendance,cLI,winning_percentage,year,total_games_played,weekday,month,night_game,al_east_opp):
    insert_into_games = ("""INSERT INTO videos (opponent,div_rank,gb,attendance,cLI,winning_percentage,
    year,total_games_played,weekday,month,night_game,al_east_opp)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (opponent,div_rank,gb,attendance,cLI,winning_percentage,year,total_games_played,weekday,month,night_game,al_east_opp)
    curr.execute(insert_into_games, row_to_insert)

def check_if_game_exists(curr, date): 
    query = ("""SELECT date FROM GAMES WHERE date = %s""")

    curr.execute(query, (date,))
    return curr.fetchone() is not None

def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['opponent'],row['div_rank'],row['gb'],row['attendance'],row['cLI'],
        row['winning_percentage'],row['year'],row['total_games_played'],row['weekday'],row['month'],row['night_game'],row['al_east_opp'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'dislike_count', 'comment_count'])
    for i, row in df.iterrows():
        if not check_if_game_exists(curr, row['video_id']):
            tmp_df = tmp_df.append(row)

    return tmp_df

In [5]:

#database credentials
host_name = 'xxxxxx'
dbname = 'xxxxx'
port = '5432'
username = 'xxxx' 
password = 'xxxxxx'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

OperationalError: could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "database-1.cjvwaetx2xw5.us-east-2.rds.amazonaws.com" (13.59.229.74) and accepting
	TCP/IP connections on port 5432?


In [ ]:
#create table

create_table(curr)

In [ ]:
#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

In [ ]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:
#view data in db table

curr.execute("SELECT * FROM GAMES")
print(curr.fetchall())